In [1]:
import os, sys
sys.path.append("./beaconrunner")

import numpy as np
import pandas as pd

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

from experiments.run import run
import experiments.templates.rl.experiment as rl_experiment

In [2]:
env = rl_experiment.beacon_env

for episode in range(1):
    state = env.reset()
    score = 0
    done = False
    states = []
    
    while not done:
        action = env.action_space.sample()
        state, balance_diff, done, info = env.step(action)
        states.append(state)
        score += balance_diff
        print('Episode: {}, States: {}, Score: {}'.format(episode, states, score))

/Users/barnabe/Documents/Research/Projects/beaconrunner/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


8 proposing block for slot 1
Episode: 0, States: [3.2050037], Score: 0.0
11 proposing block for slot 2
Episode: 0, States: [3.2050037, 5.4096503], Score: 1239345.0
7 proposing block for slot 3
Episode: 0, States: [3.2050037, 5.4096503, 11.214878], Score: 1239345.0
6 proposing block for slot 4
Episode: 0, States: [3.2050037, 5.4096503, 11.214878, 1.0656875], Score: 1239345.0
4 proposing block for slot 5
Episode: 0, States: [3.2050037, 5.4096503, 11.214878, 1.0656875, 0.14284763], Score: 1239345.0
3 proposing block for slot 6
Episode: 0, States: [3.2050037, 5.4096503, 11.214878, 1.0656875, 0.14284763, 4.384955], Score: 1239345.0
7 proposing block for slot 7
Episode: 0, States: [3.2050037, 5.4096503, 11.214878, 1.0656875, 0.14284763, 4.384955, 0.67949176], Score: 1239345.0
5 proposing block for slot 8
Episode: 0, States: [3.2050037, 5.4096503, 11.214878, 1.0656875, 0.14284763, 4.384955, 0.67949176, 10.468998], Score: 2788529.0
2 proposing block for slot 9
timestep 100 of run 1
Episode: 0,